# 関数型モデル

## ラムダ計算

- ラムダ計算では、「計算」とは「関数を適用して値を得る」ことと捉える
    - 関数: ある値を別の値に変換するための対応付け
- ラムダ計算における関数は、ラムダ式で記述される
    - ラムダ式: 数学的な関数から名前を省略して計算の本質のみを抽出した記述方法

In [6]:
"""
    (x::Int) -> x + 1

あるInt型の値 `x` を `x + 1` に変換するためのラムダ式
=> ある値に1を加算する計算
"""
# ラムダ式
(x::Int) -> x + 1

# ラムダ式は変数に「バインド（束縛）する」ことができる
## => 変数名を関数名として利用して、後から関数を適用（呼び出す）ことができるようになる
succ = (x::Int) -> x + 1

#5 (generic function with 1 method)

### 置換ルール
チューリングマシンではテープやヘッドといった「装置」を用いて計算を実行していたが、関数型モデルでは「代数的な方法」を用いて計算を実行する

- 代数的な方法: **置換ルール**
    - 式変形による計算処理
    - 変数を値に置換したり、式を同等の別式に置換したりすることで計算を実行する
    - 置換処理は、それ以上置換する要素がなくなるまで繰り返し実行される

上記の `succ` 関数の場合、計算は以下のような置換ルールに従って実行される

1. `(x) -> x + 1`
2. 変数 x を 1 で置換: x に 1 を代入する
3. `(1) -> 1 + 1`
4. `(1) -> 2` (1 + 1 -> 2 の式変形)
5. 最終的に計算結果 2 が得られる

このように置換ルールに従って式が簡単になることを、関数型プログラミングの世界では **簡約** と呼ぶ

簡約の仕組みは関数の定義が複雑になっても変わらず、置換ルールが繰り返し適用される

例えば、足し算を表す関数は以下のようになる

$$
    add(x, y) = x + y
$$

しかし、これでは足し算を定義したことにはならないため `+` 演算を使わずに足し算を定義する

基本的なアイディアは単純で、1の要素から成る2つの集合を考えて、片方の集合に要素1をすべてを移動すれば良い

```julia
# 例: 3 + 2 の定義
3 + 2

# 1. 3: [1 1 1], 2: [1 1] の集合と考える
[1 1 1] + [1 1]

# 2. 右側から左側に 1 を移動する
[1 1 1 [1]] + [(1) 1]
        ↑_______↓

# 3. 右側の集合が空になるまで 2. の処理を繰り返す
[1 1 1 1 [1]] + [(1)]
          ↑_______↓

[1 1 1 1 1] + []

# 4. 最終的に得られた左側の集合の数が足し算の結果となる
[1 1 1 1 1] -> 5
```

In [8]:
# 1の集合型
const Assemble = Vector{Val{1}}

# 2つの (1の集合型)
const AssemblePair = Pair{Assemble, Assemble}
const MaybeAssemblePair = Union{AssemblePair, Nothing}

"""
    asm(x::Int) ::Assemble

ある数値 (x) を (1の集合) に変換する
"""
asm(x::Int) ::Assemble = fill(Val(1), x)

"""
    val(a::Assemble) ::Int

ある (1の集合) aを (数値) に変換する
"""
val(a::Assemble) ::Int = length(a)

"""
    succ(pair::AssemblePair) ::AssemblePair

(2つの (1の集合)) に対して、片方の集合からもう片方の集合に 1 を移動する
上記処理後の (2つの (1の集合)) を取得する

```julia
succ(y => x) = (pop(y) => push(x, 1))
```
"""
succ(pair::AssemblePair) ::AssemblePair = (pair.first[1:end-1] => vcat(pair.second, Val(1)))

"""
    add(pair::AssemblePair) ::MaybeAssemblePair

片方の (1の集合) が空になるまで succ 関数の適用を繰り返す
* 繰り返す: 自分自身の関数適用を行う
"""
add(pair::AssemblePair) ::MaybeAssemblePair = length(pair.first) === 0 ? pair : add(succ(pair))

"""
    add(x::Int, y::Int) ::Int

足し算を行う

1. 2つの数値を (2つの (1の集合)) に変換する
2. (2つの (1の集合)) に対して succ 関数の適用を繰り返す: 簡約を行う
3. 簡約結果の (2つの (1の集合)) の内、空ではない方の (1の集合) を数値に変換する
"""
add(x::Int, y::Int) ::Int = (asm(y) => asm(x)) |> add |> (pair::AssemblePair) -> val(pair.second)

"""
test: 3 + 4 -> 7 の実行
"""
# @test: 3 + 4
# @expected: 7
add(3, 4)

7

## 関数型モデルを使うメリット

命令型モデルはアラン・チューリングによって、関数型モデルはアロンゾ・チャーチによって、いずれも1930年代にその原型が築かれた

その後、コンピュータの歴史においては、ハードウェアの構造に近い命令型モデルの方が隆盛を誇ってきた

しかし近年、CPUの処理能力の向上やメモリの巨大化など、計算機リソースが豊かになるにつれ、関数型モデル実装上の問題はほとんど解決されてきた

一方、ソフトウェアの複雑化と巨大化に伴い、以下のような関数型モデルの優れた性質が再び脚光を浴びるようになってきた

- 「置換ルール」というシンプルな計算の仕組み
    - 命令型モデルは「テープ」「ヘッド」「テーブル」など複数のルールで動く部品が多いのに対し、関数型モデルには「置換ルール」という1つのルールしかなくシンプル
    - 置換ルール自体も、一般的な数学的処理で単純でわかりやすい
- 計算過程（状態）の捉えやすさ
    - 命令型モデルでは計算の状態が「テープ」「ヘッド」「テーブル」に分散していて捉えにくいのに対し、関数型モデルでは関数の戻り値 (**蓄積変数**) がそのまま計算の状態に対応する
    - 関数型モデルでは、計算は関数から関数への伝搬で表現されるため、各関数が各状態に対応し、計算状態の明示化に成功している
- 高いモジュール性
    - 関数型モデルでは関数を値として取り扱うことができるため、処理自体を部品化できるようになる
    - これにより、どれだけ複雑な処理も単なる関数の組み合わせで表現できるため、巨大化するソフトウェアをより簡単に管理できる